In [1]:
# !wget https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images.tgz
# !wget https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/gt_files_170407.tgz

# !tar -xvzf oxbuild_images.tgz
# !tar -xvzf gt_files_170407.tgzi
# !mkdir -p collection/gtruth
# !mkdir -p collection/iamges
# !mv *.txt collection/gtruth   
# !mv *.jpg collection/iamges

In [2]:
# !pip install pytorch torchvision
# !pip install tqdm 
# !pip install faiss-cpu # pip install faiss-gpu-cu11  # For CUDA 11.x
# !    pip install transformers
# !pip install ipywidgets
# !pip install jupyterlab_widgets
# !pip install matplotlib
# !pip install opencv-python

In [3]:
import os, json
import torch
from PIL import Image
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
import faiss
import matplotlib.pyplot as plt
import cv2
from transformers import AutoModel, AutoImageProcessor

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:::", device)

Device::: cpu


In [ ]:
dinov2_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base', use_fast=True)

In [6]:
def load_image_tensor(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    return inputs["pixel_values"].to(device)

In [7]:
def extract_embedding(img_path):
    pixel_values = load_image_tensor(img_path)
    with torch.no_grad():
        embeddings = dinov2_model(pixel_values).last_hidden_state[:, 0]  # CLS token
    return embeddings.squeeze().cpu().numpy()

In [8]:
# Đường dẫn mới đến thư mục ảnh
image_dir = "collection/images"
files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")]
files.sort()  # giữ thứ tự cố định

In [9]:
# Khởi tạo FAISS Index (DINOv2-B có output 768 chiều)
index = faiss.IndexFlatL2(768)

In [ ]:
all_embeddings = {}
# Tạo embedding và lưu vào index
for img_path in tqdm(files):
    emb = extract_embedding(img_path)  # dùng hàm extract_embedding đã viết trước đó
    index.add(emb.reshape(1, -1))
    all_embeddings[img_path] = emb.tolist()

# Lưu lại index và embedding
faiss.write_index(index, "oxford_index.bin")

In [ ]:
with open("oxford_embeddings.json", "w") as f:
    json.dump(all_embeddings, f)